# Connecting with the Credo AI Governance App

Connecting with the Credo AI Governance App is straightforward using Lens. After following this tutorial you'll be up and running!

## Setup

### Find the code
This notebook can be found on [github](https://github.com/credo-ai/credoai_lens/blob/develop/docs/notebooks/governance_integration.ipynb).

### Config File
First, ensure you have a config file somewhere on your computer named ".credoconfig". The default location where Lens will look for it in your home directory (`~/.credoconfig`). The structure of the config should look like this

```
TENANT=<tenant name> # Example: credoai
API_KEY=<your api key> # Example: JSMmd26...
```

This config gives Lens API access to the Governance App.

## Data + Model Preparation (before Lens)

Some quick setup... This is all of your datascience work before assessment and integration with Credo AI.

In [ ]:
# imports for example data and model training
from credoai.data import fetch_creditdefault
from sklearn.ensemble import GradientBoostingClassifier

# Base Lens imports
import credoai.lens as cl


# set logging level to verbose
cl.set_logging_level('info')
# set default format for image displays. Change to 'png' if 'svg' is failing
%config InlineBackend.figure_formats = ['svg']

In [ ]:
data = fetch_creditdefault()
df = data['data']
df['target'] = data['target'].astype(int)

# fit model
model = GradientBoostingClassifier()
X = df.drop(columns=['SEX', 'target'])
y = df['target']
model.fit(X,y)

## Credo Governance Artifact

Lens connects to the Governance App via a `CredoGovernance` object. This specifies the IDs of the Use Case, model and data you want to associate your assessments with. The IDs are found in the url of your Use Case or model.

For instance, the Use Case ID is found here: `...credo.com/use-cases/{use_case_id}`

And the model Id is found here: `...credo.com/models/{model_id}`

The CredoGovernance object has a few functionalities.

* Retrieving an alignment spec from the Use Case (created during the aligned process). The alignment spec is either downloaded straight from the Governance App, or supplied as a json file for air gap deployment. This alignment spec can be supplemented (or overwritten) with an alignment spec you define in code and pass to Lens.
* Registering artifacts (models, datasets) to the Governance App.
* Encoding the IDs of governance objects so Lens can easily export metrics and reports.

\** **Attention** \**

The metrics and reports will _either_ be exported to the model and data specified in by `CredoGovernance` (using the `model_id` and `data_id` arguments) OR a new model and data will be registered on your CredoGovernance app automatically, which will then be used for export. You can also set the ids using the model/dataset _name_ by calling `credo_gov.set_governance_info_by_name`.

With that said, let's see how this works in practice.

In [ ]:
# New artifact for governance
credo_gov = cl.CredoGovernance(use_case_id="your-use-case-id",
                               model_id="your-model-id"
                               )

credo_model = cl.CredoModel(name='credit_default_classifier',
                            model=model)

credo_data = cl.CredoData(name='UCI-credit-default',
                          data=df,
                          sensitive_feature_key='SEX',
                          label_key='target'
                         )

In [ ]:
# specify the metrics that will be used by the Fairness assessment. 
# This spec will supplement (or overide!) whatever spec is 
# pulled down from the Governance App
alignment_spec = {
    'Fairness': {'metrics': ['precision_score', 'recall_score']}
}

## Interacting with Credo AI's Governance App from Lens

In addition to the model and data, we can pass the CredoGovernance object. This is how Lens knows which Use Case, model and/or data to interacts with. If you are running assessments on multiple models, you'll create a separate `CredoGovernance` object (and separate Lens instances) for each.

In [ ]:
from credoai.assessment import FairnessAssessment
lens = cl.Lens(model=credo_model,
               governance=credo_gov,
               assessments = [FairnessAssessment],
               data=credo_data,
               spec=alignment_spec)

In [ ]:
# After running, you just have to export and the metrics 
# will be sent to the Governance App
lens.run_assessments().export()

### Reports

Reporting is a critical feature of Lens. Your assessments must be packaged in an easily digestible way to support downstream governance. As such, Lens supports robust reporting functionality.

Lens creates a jupyter notebook report that collates all of your assessment results into one place. It then converts that notebook to HTML and sends it to Credo AI's Governance App, along with your metrics when `export` is called. The report will be associated with that use-case, as well as a model (either the model you supplied to CredoGovernance, or the model [registered](#Registering-a-model-while-assessing-it) by Lens).

In [ ]:
lens.create_report().export()

## Registering a model while assessing it

**No model on Credo AI? No Problem**

Sometimes you may not have a model or dataset registered yet in the Governance App to receive your assessments.  If you still want to log a model assessment, Lens will take care of registering a new model on Credo AI's Governance App first. 

You still need to provide a use-case ID however. You can do this by creating a `CredoGovernance` object, or just by supplying the use-case ID to the "governance" argument, as we do below.

The below will create a project called `an example model project` and log the model `an example model ` under it.

In [ ]:
from credoai.assessment import FairnessAssessment
credo_model = cl.CredoModel(name='an example model',
                            model=model)

lens = cl.Lens(model=credo_model,
               assessments = [FairnessAssessment],
               data=credo_data,
               spec=alignment_spec)

# Note! The below won't export anything unless you supply a use-case id
# A warning will be raised to inform you of this.
lens.run_assessments().create_report().export()

The governance object is created for you, and can be interrogated or used later.

In [ ]:
gov = lens.get_governance()
gov.get_info()

\** **Attention** \**


Note that if you run the above a second time, the model doesn't have to be registered again - it already was registered! Instead, Lens will find the ID for model name "an example model" and use that (this is because model names must be unique, and are only associated with one ID). If you would like to run Lens again for the same model, but store the data somewhere else you will have to either change the name of the model or explicitly supply a different model_id.

## Registering Models, Data and Use Cases

While Lens will take care of registering a model and data for you if no `model_id` or `data_id` is provided, you can also manually register models yourself using a `CredoGovernance` object.

This is helpful if you want to register a model or data without assessing it yet, if you want to run Lens multiple times, or want to have more control over the names of your artifacts. 
* First, you create a `CredoGovernance` object
* Second register your project and model
* Third, pass to Lens as normal

### Registering a Model

In [ ]:
gov = cl.CredoGovernance()
gov.register_model(model_name='my_model')

### Registering a Dataset

**Datasets** can be registered analogously.

In [ ]:
gov.register_dataset('my_data')

### Registering a Model to Use Case

**Use Cases** can also be used. If CredoGovernance has a Use Case ID, the model will automatically be associated with that Use Case ID

In [ ]:
gov = cl.CredoGovernance(use_case_id="your-use-case-id")
gov.register_model(model_name='my_model')

## Air Gap Environment

If you cannot interact with Credo AI's Governance App via the internet, this section is for you!

Instead of Lens automating the communication between your assessment environment and governance, you will instead have to download assets and upload them. 

### Getting the assessment spec
The asset that you may have to get _from_ the governance app is the assessment spec. This can be found under your use case's "Risk Assessment" tab inside "Align". Once you download the assessments spec, you can read it with a CredoGovernance object:




In [ ]:
gov = cl.CredoGovernance()
# you must explicitly retrieve the assessment spec 
# in an air gapped environment!
gov.retrieve_assessment_spec('{path to assessment spec}')

Following that, you can pass the governance object to Lens as normal.

### Exporting Results

Lens can export assessment results to file as a json object. This is generally useful if you want to store your results locally, but particularly useful if you are in an air gapped environment where you cannot access Credo AI's Governance App directly from your development environment.

Doing this only requires a one line change in our current flow. We change the `export` argument from `credoai` to a local path:


In [ ]:
lens.run_assessments().create_report().export('{path where assessments should be exported}')

Running the above will create a folder that you specified, if it doesn't exist. It will then create a json file with the assessment results and the report, which can be uploaded to Credo AI's Governance App. This json file can be uploaded by going to the assessments of your **Use-Case** and pressing the purple "upload assessment" button.